<a href="https://colab.research.google.com/github/ebamberg/TechPackersColab/blob/main/examples_LLM_fine_tuning_using_SFT_Trainer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

see: https://huggingface.co/docs/trl/main/en/sft_trainer

In [ ]:
!pip install transformers --upgrade --quiet
!pip install datasets --quiet
!pip install torch --quiet
# Attention: we have to install peft before trl
!pip install peft
!pip install -U bitsandbytes
!pip install accelerate
!pip install trl --quiet


In [ ]:
from datasets import load_dataset
from trl import SFTConfig, SFTTrainer
from peft import LoraConfig
from transformers import BitsAndBytesConfig
from transformers import AutoModelForCausalLM
from transformers import AutoTokenizer
import torch


In [ ]:
## load first 100 rows if the train features of the dataset
dataset = load_dataset("imdb", split="train[:100]")


print (type(dataset))
print (dataset)
print (dataset.features)
print (dataset.column_names)
print (dataset["text"][:3])

# an alternative way to limit the dataset to the first 100 rows
#dataset_smaller = dataset.select(range(100))
#print (dataset_smaller)

<class 'datasets.arrow_dataset.Dataset'>
Dataset({
    features: ['text', 'label'],
    num_rows: 100
})
{'text': Value(dtype='string', id=None), 'label': ClassLabel(names=['neg', 'pos'], id=None)}
['text', 'label']
['I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with

In [ ]:
llmmodels =["facebook/opt-350m","EleutherAI/gpt-neo-125m","meta-llama/Meta-Llama-3.1-8B-Instruct"]

modelid=llmmodels[2]

In [ ]:
sft_config = SFTConfig(
    dataset_text_field="text",
    max_seq_length=512,
    output_dir="/tmp",
 #   model_init_kwargs={
 #       "torch_dtype": "bfloat16",
 #   },
)

quantization_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_compute_dtype=torch.bfloat16

)

peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

training_args = TrainingArguments(
    per_device_train_batch_size=16,
    learning_rate=2e-4,
    lr_scheduler_type="cosine",
    num_train_epochs=3,
    gradient_accumulation_steps=2, # simulate larger batch sizes
)

model = AutoModelForCausalLM.from_pretrained(
    modelid,
 #   load_in_4bit=True,
 #    model_kwargs={"torch_dtype": torch.bfloat16},
    quantization_config=quantization_config,
    device_map="auto",
)
tokenizer = AutoTokenizer.from_pretrained(modelid)

def prompt_generator(row):
  print (row)
  return ("Below is an instruction that describes a task. "
            "Write a response that appropriately completes the request.\n\n"
            "### Instruction:\n{text}\n\n### Response:\n{label}").format_map(row)


trainer = SFTTrainer(
    model,
    train_dataset=dataset,
    args=sft_config,
    peft_config=peft_config,
    max_seq_length=1024,
    arg=training_args,
    formatting_func=prompt_generator
)
trainstats=trainer.train()


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 64.00 MiB. GPU 

In [ ]:
print (trainstats)

TrainOutput(global_step=39, training_loss=3.592804346329127, metrics={'train_runtime': 71.9668, 'train_samples_per_second': 4.169, 'train_steps_per_second': 0.542, 'total_flos': 265355674976256.0, 'train_loss': 3.592804346329127, 'epoch': 3.0})


Example prediction

In [ ]:
from transformers import pipeline
generator = pipeline('text-generation', model=model, tokenizer=tokenizer)
generator("What can you tell me about I AM CURIOUS-YELLOW ?")

[{'generated_text': 'What can you tell me about I AM CURIOUS-YELLOW ?\n\n\n'}]